### The following Notebook extracts the UME cohort of patients with AL types from the FHIR database of UME.

***The cohort is defined as follows:***

- Only patients with ALL, AML and APL for the main diagnosis, based on ICD-10 codes.
- Only patients with Aplast. Anemia, MDS, CML and MCL for differential diagnosis, based on ICD-10 codes
- Diagnosis not older than 2015-01-01
- Only patients age 18 years and older
- Diagnosis must be discharge diagnosis or postoperative diagnosis
- Ambulatory hospital encounters are exlcuded
- Only laboratory values recorded within 72 hours after hospital admission are included. If multiple laboratory values of the same code are available, the first recorded laboratory value is used

For querying data from the FHIR-database, FHIR-Pyrate was used, see:<br>
https://github.com/UMEssen/FHIR-PYrate

***This Python script executes the following steps:***

1. Authenticate to FHIR-Server
2. Search FHIR-resource Condition, filtered by defined ICD-10 codes and date >= 2015-01
3. Filter for diagnosis made at discharge or postoperative
4. Search FHIR-resource Encounter, filtered by discharge and postoperative diagnosis
5. Search FHIR-resource Observation for laboratory values, filtered by inpatient encounters
6. Filter laboratory values for relevant codes and only keep first recorded laboratory values
7. Search FHIR-resource Patient for birth date and sex, filtered by patients for which laboratory values were recorded


In [2]:
import os

import pandas as pd
from dotenv import load_dotenv
from fhir_pyrate import Ahoy
from fhir_pyrate import Pirate

/Users/tillrostalski/Git/case-study-ml-acute-leukemia/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Set up the environment
load_dotenv()
BASIC_AUTH = os.environ["BASIC_AUTH"]
REFRESH_AUTH = os.environ["REFRESH_AUTH"]
BASE_URL = os.environ["BASE_URL"]

# Define path to store the data
DATA_PATH = os.path.join(os.path.abspath(''), "..", "data")

# Authenticate against the FHIR server
auth = Ahoy(
    auth_type="token",
    auth_method="env",
    auth_url=BASIC_AUTH,
    refresh_url=REFRESH_AUTH,
)

# Initiate Pirate
search = Pirate(
    auth=auth,
    base_url=BASE_URL,
    print_request_url=False
)

In [8]:
# ICD-10 codes for main AL types and differential diagnoses
conditions_al = pd.DataFrame([
    {'icd-10': 'C92.40'},{'icd-10': 'C92.4'},
    {'icd-10': 'C92.4%2B'},{'icd-10': 'C92.40%2B'},
    {'icd-10': 'C92.00'},{'icd-10': 'C92.0'},
    {'icd-10': 'C92.00%2B'},{'icd-10': 'C92.0%2B'},
    {'icd-10': 'C91.00'},{'icd-10': 'C91.0'},
    {'icd-10': 'C91.00%2B'},{'icd-10': 'C91.0%2B'},
    {'icd-10': 'C92.10'},{'icd-10': 'C92.1'},
    {'icd-10': 'C92.10%2B'},{'icd-10': 'C92.1%2B'},
    {'icd-10': 'D46.90'},{'icd-10': 'D46.9'},
    {'icd-10': 'D46.90%2B'},{'icd-10': 'D46.9%2B'},
    {'icd-10': 'D61.90'},{'icd-10': 'D61.9'},
    {'icd-10': 'D61.90%2B'},{'icd-10': 'D61.9%2B'}
    {'icd-10': 'C83.10'},{'icd-10': 'C83.1'},
    {'icd-10': 'C83.1%2B'},{'icd-10': 'C83.10%2B'},
    {'icd-10': 'C83.11'},{'icd-10': 'C83.12'},
    {'icd-10': 'C83.11%2B'},{'icd-10': 'C83.12%2B'},
    {'icd-10': 'C83.13'},{'icd-10': 'C83.14'},
    {'icd-10': 'C83.13%2B'},{'icd-10': 'C83.14%2B'},
    {'icd-10': 'C83.15'},{'icd-10': 'C83.16'},
    {'icd-10': 'C83.15%2B'},{'icd-10': 'C83.16%2B'},
    {'icd-10': 'C83.17'},{'icd-10': 'C83.18'},
    {'icd-10': 'C83.17%2B'},{'icd-10': 'C83.18%2B'},
    {'icd-10': 'C83.19'},{'icd-10': 'C83.19%2B'},
])

In [10]:
# Search FHIR-resource Condition, filtered by defined ICD-10 codes and date >= 2015-01
dr_bundles = search.trade_rows_for_bundles(
  conditions_al,
  resource_type="Condition",
  request_params={"_count": "10000","_sort":'_id',"recorded-date": "ge2015-01"},
  df_constraints={"code": "icd-10"}
)
conditions_df = search.bundles_to_dataframe(
    bundles=dr_bundles)
conditions_df.to_csv(f"{DATA_PATH}/conditions_al_types.csv",sep=';',index=False)

# Filter for types discharge and postoperative diagnosis
cond_type = ['ENT', 'POP']
conditions_df_dis = conditions_df[conditions_df['category_0_coding_0_code'].isin(cond_type)]
conditions_df_dis.to_csv(f"{DATA_PATH}/conditions_al_types_dis_pop.csv",sep=';',index=False)


Query Rows (Condition): 100%|██████████| 24/24 [01:48<00:00,  4.54s/it]


In [ ]:
# Search FHIR-resource Encounter, filtered by discharge and postoperative diagnosis
encounters = conditions_df_dis['encounter_reference'].dropna().drop_duplicates().reset_index(drop=True).str.split('Encounter/').str[-1].to_frame(name='encounter_reference')

dr_bundles = search.trade_rows_for_bundles(
  encounters,
  resource_type="Encounter",
  request_params={"_count": "10000","_sort":'_id'},
  df_constraints={"_id": "encounter_reference"}
)

encounters_df = search.bundles_to_dataframe(
    bundles=dr_bundles)
encounters_df.to_csv(f"{DATA_PATH}/encounters.csv",sep=';',index=False)

# Exclude ambulatory encounters
idx_drop = encounters_df[encounters_df['class_code'] == 'AMB'].index
encounters_df_imp = encounters_df.drop(idx_drop)
encounters_df_imp.to_csv(f"{DATA_PATH}/encounters_imp.csv",sep=';',index=False)


In [ ]:
# Search FHIR-resource Observation for laboratory values, filtered by inpatient encounters
encounters_df_imp['id'] = encounters_df_imp['id'].apply(lambda x: 'Encounter/' + x)

dr_bundles = search.trade_rows_for_bundles(
  encounters_df_imp,
  resource_type="Observation",
  request_params={"category": "laboratory","_sort":"date","_count": "10000"},
  df_constraints={"encounter": "id"}
)
labs_df = search.bundles_to_dataframe(
    bundles=dr_bundles)

labs_df.to_csv(f"{DATA_PATH}/labs.csv",sep=';',index=False)


In [ ]:
# Keep only lab values relevant to AIPAL
labs_filter = ["zMCV", "zMCHC", "zTHRA", "zLEUA", "zLYM#", "zMON#", "zTPZCS", "zFIBRCS", "zLDH", "zBLAM"]
lab_df_subset = labs_df[labs_df['code_coding_0_code'].isin(labs_filter)]
lab_df_subset.to_csv(f"{DATA_PATH}/labs_aipal.csv",sep=';', index=False)

# Get only the first recorded lab value per encounter
# Check datetime type
lab_df_subset['effectiveDateTime'] = pd.to_datetime(lab_df_subset['effectiveDateTime'])
lab_values_first = lab_df_subset.dropna(subset=['effectiveDateTime'])

# Group by encounter_reference and code_coding_0_code and get the indices of minimal effectiveDateTime
idx = lab_values_first.groupby(['encounter_reference', 'code_coding_0_code'])['effectiveDateTime'].idxmin()

# Filter the DataFrame using the indices
lab_values_first = lab_values_first.loc[idx]
lab_values_first = lab_values_first.reset_index(drop=True)
lab_values_first.to_csv(f"{DATA_PATH}/labs_aipal_first.csv",sep=';', index=False)

In [ ]:
# Search FHIR-resource Patient, filtered by patients for which laboratory values are available
lab_values_first['subject_id'] = lab_values_first['subject_reference'].str.replace('Patient/', '')
lab_values_unique = lab_values_first[['subject_id']].drop_duplicates().reset_index(drop=True)

dr_bundles = search.trade_rows_for_bundles(
  lab_values_first,
  resource_type="Patient",
  request_params={"_count": "10000"},
  df_constraints={"_id": "subject_id"}
)

patients_df = search.bundles_to_dataframe(
    bundles=dr_bundles,
    fhir_paths=[("subject_reference", "id"), ("birth_date", "birthDate"),"gender"]
)
patients_df.to_csv(f"{DATA_PATH}/patients_aipal.csv",sep=';', index=False)